# Image Posterization using K-means Clustering Algorithm

In [ ]:
#import PIL as pil
from PIL import Image
from skimage import io
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
import joblib
import time
import os 


Image compression will be in a method that takes an image and a percentage of pixels to sample. The outputs will be the k selected and the compressed image.

In [ ]:
def smartPost(oldImage, k, percentage, fileName):
    image = oldImage
    pixels = image.load()

    random.seed(a=100)
    #k = 100   #we would have a loop and iterate k

    skip = int(1 / np.sqrt(percentage))
    count = 0
    size = np.ceil(image.size[0] / skip) * np.ceil(image.size[1] / skip)
    df_pixels = pd.DataFrame(columns=list('RGB'), index = np.arange(size))

    timeBeforeSample = time.time()
    for i in range(0, image.size[0], skip):
        for j in range(0, image.size[1], skip):
            i_ = i + random.randint(-3,3)
            if i_>=image.size[0]:
                i_ = image.size[0] - 1
            if i_<0:
                i_ = 0
            j_ = j + random.randint(-3,3)
            if j_>=image.size[1]:
                j_ = image.size[1] - 1
            if j_<0:
                j_ = 0
                
            df_pixels.iloc[[count]] = [[pixels[i_,j_][0], pixels[i_,j_][1], pixels[i_,j_][2]]]
            count = count + 1
    timeAfterSample = time.time()
    sampleTime = timeAfterSample - timeBeforeSample
    kmeans = KMeans(n_clusters=k, random_state=1, n_init=1) #k
    kmeans.fit(df_pixels) #run on df_pixels
    
    timeAfterKmeansFit = time.time()

    KMeansTime = timeAfterKmeansFit - timeAfterSample

    joblib.dump(kmeans, 'kmeans.pkl')
    kmeans_loaded = joblib.load('kmeans.pkl')
    kmeans_loaded

    timeAfterPickle = time.time()
    PickleTime = timeAfterPickle - timeAfterKmeansFit



    for i in range(0, image.size[0]):
        for j in range(0, image.size[1]):
            predicted = kmeans_loaded.predict([[pixels[i,j][0], pixels[i,j][1], pixels[i,j][2]]])
            pixels[i,j] = (round(kmeans.cluster_centers_[predicted][0][0]), round(kmeans.cluster_centers_[predicted][0][1]), round(kmeans.cluster_centers_[predicted][0][2]))
            #pixels[i,j] = (0,0,0)

    timeAfterCompression = time.time()
    CompressionTime = timeAfterCompression - timeAfterPickle

    saveLocation = fileName.replace("Inputs","Outputs")

    image.save(saveLocation, format="png")

    timeAfterSave = time.time()
    SaveTime = timeAfterSave - timeAfterCompression
    return sampleTime, KMeansTime, PickleTime, CompressionTime, SaveTime

In [ ]:
#percentage = 0.02
#
#currDir = 'Inputs/Explorations' 
#distortions = []
#for filename in os.listdir(currDir):
#    if filename.endswith(".png"):
#        currImg = Image.open(str(currDir)+'/'+str(filename))
#        for k in range(2,20):
#            distortions.append(smartPost(currImg, k, percentage, str(currDir)+'/'+str(filename)))
#
#graph = plt.figure(figsize=(15,5))
#graph.plot(range(2,20), distortions)
#graph.title('Elbow Curve')

In [ ]:
percentage = 0.01
k=1 #remove later

currDir = 'Inputs/Landscapes' 
for filename in os.listdir(currDir):
   if filename.endswith(".png"):
       currImg = Image.open(str(currDir)+'/'+str(filename))
       t0, t1, t2, t3 ,t4 = smartPost(currImg, k, percentage, str(currDir)+'/'+str(filename))

print('t0, t1, t2, t3, t4 ' +str(t0)+' '+str(t1)+' '+str(t2)+' '+str(t3)+' '+str(t4)+'\n')

#percentage = 1
#k=100 #remove later

currDir = 'Inputs/Animals' 
for filename in os.listdir(currDir):
   if filename.endswith(".png"):
       currImg = Image.open(str(currDir)+'/'+str(filename))
       t0, t1, t2, t3 ,t4 = smartPost(currImg, k, percentage, str(currDir)+'/'+str(filename))

print('t0, t1, t2, t3, t4 ' +str(t0)+' '+str(t1)+' '+str(t2)+' '+str(t3)+' '+str(t4)+'\n')

percentage = 0.10
k=10 #remove later

currDir = 'Inputs/Cars' 
for filename in os.listdir(currDir):
    if filename.endswith(".png"):
        currImg = Image.open(str(currDir)+'/'+str(filename))
        t0, t1, t2, t3 ,t4 = smartPost(currImg, k, percentage, str(currDir)+'/'+str(filename))

print('t0, t1, t2, t3, t4 ' +str(t0)+' '+str(t1)+' '+str(t2)+' '+str(t3)+' '+str(t4)+'\n')